# Reinforcement Learning Project

#### Adrien DANEL & Slimane THABET

This notebook shows our work related to this paper : [**Can Deep Reinforcement Learning Solve Erdos-Selfridge-Spencer Games?**](https://arxiv.org/abs/1711.02301)

### Paper description

The paper points out that developing learning algorithms that are robust across tasks and policy representations remains a
challenge. While standard benchmarks like MuJoCo and Atari provide rich settings for experimentation, but the specifics of the underlying environments differ from each other in multiple ways, and thus determining the principles underlying any particular form of sub-optimal behavior is difficult. To evaluate the success of the alorithms we try on those environments, we usually uses **high scores** to determine those which are more performant, but it would be more interesting to look at **optimal behavior**, which is unfortunately not fully characterized and generally complex on this type of environment.

The article proposes an new environment to better evaluate generalization, it is based on the work of Erdos and Selfridge, and Spencer. Erdos-Selfridge-Spencer (ESS) games are turn-based games where two players play adversarly. The cool thing is that **optimal behavior** (or strategy) can be defined by **potential functions** derived from conditional expectations over random future play.

They chose to focus on a very well-known game type in the Reinforcement Learning, the **attacker-defender game**.

The following picture, taken from the paper, explains how a turn works :

<img src='schema_ess.PNG'>

### Environment specificities

ESS games offer a nice field to experiment with RL techniques, because there exists several theoretical results which make evaluation of performance easier.

Let us start with a definition.

**Definition : potential function**

Let us consider a state $\mathcal{S} = (n_1, ..., n_k)$ where $n_i$ is the number of pieces in level $i$. We define then the *potential* of $\mathcal{S}$, noted $\phi(\mathcal{S})$ by the following function:

$$\phi(\mathcal{S}) = \sum_{i=1}^{K}n_i2^{-(K-i)}$$

The potential is a key metric for results and strategies of ESS games.


The possible outcomes and optimal strategies are in the following theorems:

**Theorem 1**

Let an instance of the ESS game with $N$ pieces and $K$ levels randomly placed on the board, and $\mathcal{S}_0$ the initial state.

- If $\phi(\mathcal{S}_0)<1$, the defender can always win
- If $\phi(\mathcal{S}_0)\geq1$, the attacker can always win

**Theorem 2**

The optimal strategy for the defender is the following:

- For the two proposed parts $A$ and $B$, compute $\phi(A)$ and $\phi(B)$.
- Eliminate the part with the highest potential.


For the rest of the work we will call *random* policy the policy where the defender chooses randomly between the sets $A$ and $B$, and the *optimal* policy the policy where it plays in an optimal way.

#### What we chose to do

While the authors decided to create an environment based on the ESS using **[gym](https://gym.openai.com/)** (an **openAI** library for developing and comparing reinforcement learning algorithms on famous environments like **Cartpole** or **Pong**) and compare openAI algorithms on the new environment, we decided to do somehting different.

We didn't want to spend time on recreating a gym-compatible ESS environment, and then use the already-made RL algorithms like DQN, PPO or A2C. We felt like it was more interesting to develop our own RL algorithm on an ESS environment that we would also create from scratch.

## Algorithms

We implemented the Monte-Carlo method for policy evaluation and policy control without exploring start. The general algorithm is the following :

<img src='MC_RL_algo.png' width="600" height="400">

We slightly adapted this algorithm for the purpose of the game. For instance, it was difficult to enumerate every pair (state,action) from the begining due to the high number of possibilities. Furthermore, only 2 actions are available at every round for the defender, much less that all the theoretical pairs.

The changes were then the following :

- Not initializing Q for every $(s,a)$, but filled at the end of each game
- For updating $\pi$:
    1. Look if the pairs $(s,a)$ and $(s,b)$ are in the table
        * If both are in the table chose $argmax_{a,b} \{Q(s,a), Q(s,b)\}$
        * If one is missing play chose the one who is present
        * If both are missing, chose one randomly
    2. Play the chosen action with probability $1-\epsilon$ and the other with probability $\epsilon$

#### Code execution

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 27 14:49:15 2020

@author: slimane
"""

import numpy as np
import random
import pickle
from time import time
from ESS import Game
import matplotlib.pyplot as plt

    
def hash_action(action):
    '''Returns the action as a string to make a dictionnary key'''
    return str(action)
       
 
        
def policy(Q, state, split_a, split_b, epsilon=0.1, play_random=False, play_optimal=False):
    '''Play the policy'''
    
    #If we play random, choose a random part
    if play_random:
        if np.random.uniform()>0.5:
            return split_a
        else:
            return split_b
    #If we play optimal, return the part with highest potential  
    if play_optimal:
        weights = 0.5 ** (np.arange(len(split_a)))
        pot_a = np.sum(weights*split_a)
        pot_b = np.sum(weights*split_b)
        if pot_a>=pot_b:
            return split_a
        else:
            return split_b
        
    #Get the keys of the actions to look in the table
    key_a = hash_action(split_a)
    key_b = hash_action(split_b)
    bigger = split_a
    smaller = split_b
    if np.random.uniform()>0.5:
        bigger = split_b
        smaller = split_a
    
    if state in Q.keys():
        if key_a in Q[state].keys():
            if key_b in Q[state].keys():
                if Q[state][key_a] >= Q[state][key_b]:
                    bigger = split_a
                    smaller = split_b
                else:
                    bigger = split_b
                    smaller = split_a
            else:
                bigger = split_a
                smaller = split_b
        else:
            if key_b in Q[state].keys():
                bigger = split_a
                smaller = split_b
                
    #Whatever choice, flip it with epsilon probability          
    if np.random.uniform()>epsilon:
        return bigger
    else:
        return smaller
    
#Initialize the game
game = Game(8, 500, init_bottom=True)

Q = {}
returns = {}

gamma = 0.9

#Number of Monte-Carlo iterations
N = 30000

defender_wins = []#Evaluation of the policy during the training

for i in range(N):
    
    list_of_states = []
    list_of_actions = []
    list_of_rewards = []
    list_of_returns = []
    winner = None
    game.reset(init_bottom=True)
    
    #play the game
    while winner != 'attacker' and winner != 'defender':
        a,b = game.play_attacker()
        action = policy(Q, game.getHash(), a, b, epsilon=0.3)
        assert (action==a).all() or (action==b).all()
        list_of_states.append(game.getHash())
        list_of_actions.append(hash_action(action))
        game.play_defender(action)
        winner = game.winner()
        if winner =='attacker':
            list_of_rewards.append(-1)
        elif winner=='defender':
            list_of_rewards.append(1)
        else:
            list_of_rewards.append(0)
    
    assert len(list_of_states) == len(list_of_rewards)
    
    #compute the returns
    n = len(list_of_states)      
    list_of_returns.append(0)
    for j in range(1,n+1):
        list_of_returns.append(list_of_rewards[n-j] + gamma * list_of_returns[j-1])
    list_of_returns.reverse()
    
    assert len(list_of_states) == len(list_of_returns)-1
    assert list_of_returns[-1]==0
    #add returns to dictionnary
    for j,s in enumerate(list_of_states):
        a = list_of_actions[j]
        if s in returns.keys():
            if a in returns[s].keys():
                returns[s][a].append(list_of_returns[j])
            else:
                returns[s][a] = [list_of_returns[j]]
        else:
            returns[s] = {}
            returns[s][a] = [list_of_returns[j]]
            
    #Every 500 steps, update the policy       
    if i%500==0:
        for s in returns.keys():
            Q[s] = dict()
            for a in returns[s].keys():
                Q[s][a] = np.mean(returns[s][a])
        defender_wins.append(np.sum(evaluate_policy(init_bottom=True)))
        

        
def evaluate_policy(N=1000, random=False, optimal=False, init_bottom=False):
    winner_defender = np.zeros(N)
    for i in range(N):
        game.reset(init_bottom=init_bottom)
        winner=None
        while winner != 'attacker' and winner != 'defender':
            a,b = game.play_attacker()
            action = policy(Q, game.getHash(), a, b, play_random=random, play_optimal=optimal)
            assert (action==a).all() or (action==b).all()
            game.play_defender(action)
            winner = game.winner()
            if winner=='defender':
                winner_defender[i] = 1
    return winner_defender
        

#Check the result of training by comparing to random and optimal policy
print("In random case, defender wins ", np.sum(evaluate_policy(random=True, init_bottom=True))/10, "% times")  
print("After training, defender wins ", np.sum(evaluate_policy(init_bottom=True))/10, "% times")
print("In optimal case, defender wins ", np.sum(evaluate_policy(optimal=True, init_bottom=True))/10, "% times")





#### Results

- normal
- cas ou le defendeur PEUT gagner ?

<img src='training_plot_8_500_bottom.svg?sanitize=true' width="600" height="400">